<a href="https://colab.research.google.com/github/vijishmadhavan/App/blob/master/App2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# classifier
import urllib.request
from fastai.vision import *

defaults.device = torch.device('cpu')

MODEL_URL = "https://www.dropbox.com/s/3s9x9zllbegpilx/Aesthetic.pkl?dl=0"
urllib.request.urlretrieve(MODEL_URL, "Aesthetic.pkl")
path = Path(".")

learner = load_learner(path, 'Aesthetic.pkl')

def classify(url):
  urllib.request.urlretrieve(url, "image.jpg")
  img = open_image("image.jpg")

  pred_class,_,probs = learner.predict(img)
  return pred_class.obj, learner.data.classes, probs.tolist()

In [ ]:
# interface
import ipywidgets as widgets
from IPython.display import display

def classify_on_click(btn):
  htmlResult.value = "<h3>Analysing...</h3>"
  display_image(txtURL.value)
  pred_class,classes,probs = classify(txtURL.value)
  pred_class = pred_class.replace("-aesthetic", "") #because i should have named my classes better...

  html = "<div align='center'>"
  html += "<h3>This is" + pred_class + "</h3>"
  html += "Probilities: " 
  html += classes[0] + "=" + "{:1.2f}".format(probs[0]) + ", "
  html += classes[1] + "=" + "{:1.2f}".format(probs[1]) 
  html += "</div>"
  htmlResult.value = html

def display_image(url):
  htmlImage.value= "<img src='" + url + "' width='300'>"

DEFAULT_IMG = "https://expertphotography.b-cdn.net/wp-content/uploads/2018/01/twilight-photography-harbour-lights.jpg"

htmlImage = widgets.HTML(value="")
display_image(DEFAULT_IMG)

txtURL = widgets.Text(description="URL:", placeholder="Test the below given pictures!!")
txtURL.layout.width = "500px"
btnClassify = widgets.Button(description="Classify")
btnClassify.on_click(classify_on_click)
boxH = widgets.HBox([txtURL, btnClassify])

htmlResult = widgets.HTML(value="<h3>I fed clowns to a fastai convnet. They tasted funny.</h3>")
htmlResult.layout.margin = "0px 0px 40px 0px"

boxV = widgets.VBox([htmlImage, boxH, htmlResult])

boxV.layout.align_items = "center"
display(boxV)

# give them some examples to play with without getting urls
def example_on_click(btn):
  txtURL.value = btn.tag
  classify_on_click(btnClassify)

def create_example(url):
  img = widgets.HTML("<img src='" + url + "' height='100'>")
  btn = widgets.Button(description="Test")
  btn.tag = url
  btn.on_click(example_on_click)
  vbox = widgets.VBox([img,btn])
  vbox.layout.align_items = "center"
  return vbox

urls = [
  "https://www.shutterstock.com/blog/wp-content/uploads/sites/5/2018/03/1-24.jpg", 
  "https://iso.500px.com/wp-content/uploads/2015/12/georgy.jpg",
  "https://cf.ltkcdn.net/weddings/images/std-xs/138107-338x400-wedpose3.jpg",
  "https://i2-prod.mirror.co.uk/incoming/article13973295.ece/ALTERNATES/s615/0_PAY-BLP_03_MC_DONAGH_WEDDING_SCAM.jpg",
  "https://i.ytimg.com/vi/FpIdseKHAq0/maxresdefault.jpg"
  
  ]
  
items = []
for url in urls:
    items.append(create_example(url))

grid = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 33%)"))
display(grid)